# RAG 运维知识库示例
借助 LangChain TextSplitter，我们可以先将 markdown 格式的运维知识库进行拆分，然后进行 Embedding 并存储在本地向量数据库中。当用户提问时，将用户的问题也进行 Embedding，然后通过向量检索的方式找到最相似的知识库内容，并将内容返回给大模型，然后再进行问答。

In [1]:
! pip install -qU langchain-openai langchain langchain_community langchainhub
! pip install chromadb==0.5.3
# 设置 OPENAI API KEY
# # export OPENAI_API_KEY=sk-...


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from langchain import hub as langchain_hub
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
#from langchain_chroma import Chroma
from langchain_community.vectorstores.chroma import Chroma

In [21]:
# 读取 ./data/data.md 文件作为运维知识库
file_path = os.path.join('data', 'data.md')
with open(file_path, 'r', encoding='utf-8') as file:
    docs_string = file.read()

# Split the document into chunks base on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = text_splitter.split_text(docs_string)
print("Length of splits: " + str(len(splits)))
print(splits)

Length of splits: 11
[Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '1. 系统监控指标'}, page_content='- **交易量**: 监控每秒交易数，确保系统承载能力。\n- **响应时间**: 监控交易的平均响应时间，确保服务性能。\n- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.1 交易失败'}, page_content='- **问题描述**: 用户发起支付后，交易未能成功完成。\n- **可能原因**:\n- 网络延迟或中断\n- 支付网关服务异常\n- **解决办法**:\n- 检查网络连接，确保支付服务的网络畅通。\n- 检查支付网关服务状态，重启服务或联系服务提供商。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.2 响应时间过长'}, page_content='- **问题描述**: 用户支付请求处理时间超过正常范围。\n- **可能原因**:\n- 系统资源不足\n- 数据库查询效率低下\n- 外部服务响应慢\n- **解决办法**:\n- 增加系统资源，如CPU、内存。\n- 优化数据库查询，使用索引，减少复杂查询。\n- 与外部服务提供商沟通，优化接口性能。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.3 系统宕机'}, page_content='- **问题描述**: 支付系统完全无法访问或服务中断。\n- **可能原因**:\n- 主机硬件故障\n- 系统软件崩溃\n- 网络设备故障\n- **解决办法**:\n- 快速切换到备用服务器。\n- 检查系统日志，定位问题原因。\n- 联系硬件供应商，进行故障排查和修复。'), Document(metadata={'Head

In [4]:
# 将运维知识库的每一块文本向量化（Embedding）
embedding = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key="sk-***",openai_api_base="https://api.apiyi.com/v1")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding,persist_directory="./chroma_langchain_db")
vectorstore.persist()

None


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [24]:
# 文本相似性检索
results = vectorstore.similarity_search_with_score(
    "payment 服务",
    k=1,
)
for res, score in results:
    print(f"* [相似性={score:3f}] {res.page_content} [{res.metadata}]")

* [相似性=0.877543] - **payment**: 小张，联系方式：18888888888
- **payment_gateway**: 小王，联系方式：18888888889
- **payment_callback**: 小李，联系方式：18888888890 [{'Header 1': '支付系统运维知识库', 'Header 2': '8. 业务负责人'}]


In [26]:
# 向量检索
results = vectorstore.similarity_search_by_vector_with_relevance_scores(
    embedding=embedding.embed_query("payment 服务是谁维护的？"), k=1
)
for doc, score in results:
    print(f"* [相似性={score:3f}] {doc.page_content} [{doc.metadata}]")

* [相似性=1.088376] - **payment**: 小张，联系方式：18888888888
- **payment_gateway**: 小王，联系方式：18888888889
- **payment_callback**: 小李，联系方式：18888888890 [{'Header 1': '支付系统运维知识库', 'Header 2': '8. 业务负责人'}]


In [27]:
# 向量检索时会把 input Embedding 之后转成向量
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="payment 服务是谁维护的？",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

[0.013615109957754612, -0.03324897587299347, 0.055096786469221115, 0.008762987330555916, -0.004749380052089691, -0.0022156040649861097, -0.015192712657153606, 0.051093120127916336, -0.0065258401446044445, -0.058543648570775986, 0.09248199313879013, 0.004560465458780527, 0.010479790158569813, -0.044384993612766266, -0.025003015995025635, 0.00717212725430727, -0.03065056912600994, 0.03884349763393402, -0.04202521964907646, -0.017632031813263893, 0.003385548945516348, 0.03955938667058945, 0.005349266808480024, -0.01219659298658371, -0.041600991040468216, 0.013654882088303566, -0.04878637194633484, -0.041760075837373734, 0.02243112586438656, -0.0642707422375679, 0.06358136981725693, -0.029749082401394844, 0.05615735799074173, 0.030385425314307213, 0.03810109570622444, -0.002717719180509448, -0.005452009849250317, -0.02961651049554348, 0.017579002305865288, -0.006880469620227814, -0.023067470639944077, -0.05578615888953209, -0.003446863265708089, -0.022736040875315666, -0.006167897023260593

In [31]:
# 使用运维知识库进行检索生成
retriever = vectorstore.as_retriever()
# 使用 LangChain 提供的 RAG Prompt，来源于：https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=989ad331-949f-4bac-9694-660074a208a7
"""
这是 RAG Prompt 原文：

human

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:
"""
prompt = langchain_hub.pull("rlm/rag-prompt")

def format_docs(docs):
    print("匹配到的运维知识库片段：\n", "\n\n".join(doc.page_content for doc in docs))
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(model="gpt-4o-mini")
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 问题将会被 Embedding 转成向量，然后再从向量数据库检索
res = rag_chain.invoke("payment 服务是谁维护的？")
# RAG 无法回答的问题
# res = rag_chain.invoke("谁负责的服务最多？")
print("\n\nLLM 回答：", res)

匹配到的运维知识库片段：
 - **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **问题描述**: 用户支付请求处理时间超过正常范围。
- **可能原因**:
- 系统资源不足
- 数据库查询效率低下
- 外部服务响应慢
- **解决办法**:
- 增加系统资源，如CPU、内存。
- 优化数据库查询，使用索引，减少复杂查询。
- 与外部服务提供商沟通，优化接口性能。

- **问题描述**: 用户支付请求处理时间超过正常范围。
- **可能原因**:
- 系统资源不足
- 数据库查询效率低下
- 外部服务响应慢
- **解决办法**:
- 增加系统资源，如CPU、内存。
- 优化数据库查询，使用索引，减少复杂查询。
- 与外部服务提供商沟通，优化接口性能。


LLM 回答： 根据上下文，无法确定具体负责的服务最多。所提供的信息主要集中在监控和问题解决方面，并未指明具体责任或服务数量。
